<a href="https://colab.research.google.com/github/Htets-Corner/SYNTHBUSTER_RAISE-1k/blob/main/syn_real_mobilevit050.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
# Step 0: Data Preparation

# 1. Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# 2. Define dataset path
import os

# Update this path if needed
dataset_path = "/content/drive/MyDrive/Dataset"

train_dir = os.path.join(dataset_path, "train")
test_dir  = os.path.join(dataset_path, "test")

print("Train path:", train_dir)
print("Test path:", test_dir)

# 3. Import necessary libraries
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# 4. Define image transformations (resize, normalization)
# MobileViT usually works with 256x256 or 224x224 input
image_size = 256

transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.5, 0.5, 0.5],   # normalize to [-1, 1]
        std=[0.5, 0.5, 0.5]
    )
])

# 5. Load train and test datasets
#train_dataset = datasets.ImageFolder(root=train_dir, transform=transform)
#test_dataset  = datasets.ImageFolder(root=test_dir, transform=transform)

from torchvision.datasets import ImageFolder

valid_exts = ('.jpg', '.jpeg', '.png', '.PNG', '.bmp', '.tif', '.tiff', '.webp')

train_dataset = ImageFolder(
    root=train_dir,
    transform=transform,
    is_valid_file=lambda path: path.lower().endswith(valid_exts)
)

test_dataset = ImageFolder(
    root=test_dir,
    transform=transform,
    is_valid_file=lambda path: path.lower().endswith(valid_exts)
)


# 6. Create DataLoaders
#batch_size = 32
batch_size = 16

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
test_loader  = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

# 7. Check class labels
classes = train_dataset.classes
print("Classes:", classes)
print("Train size:", len(train_dataset))
print("Test size:", len(test_dataset))


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Train path: /content/drive/MyDrive/Dataset/train
Test path: /content/drive/MyDrive/Dataset/test
Classes: ['ai', 'real']
Train size: 3199
Test size: 800


In [5]:
import os

print("Train/AI files:", len(os.listdir(train_dir + "/ai")))
print("Train/Real files:", len(os.listdir(train_dir + "/real")))
print("Test/AI files:", len(os.listdir(test_dir + "/ai")))
print("Test/Real files:", len(os.listdir(test_dir + "/real")))

# Show first 5 files in each folder
print("Sample AI:", os.listdir(train_dir + "/ai")[:5])
print("Sample Real:", os.listdir(train_dir + "/real")[:5])


Train/AI files: 2400
Train/Real files: 799
Test/AI files: 600
Test/Real files: 200
Sample AI: ['stable-diffusion-2_r179bb406t.png', 'stable-diffusion-2_r0e965ba7t.png', 'firefly_r03f70ccdt.png', 'stable-diffusion-xl_r164c1e13t.png', 'firefly_r138ad247t.png']
Sample Real: ['r15c91802t.png', 'r0db6cc31t.png', 'r1aa53167t.png', 'r05d9d749t.png', 'r13de1e12t.png']


In [7]:
# Run this first cell in Colab
!pip install -q timm        # recommended: timm includes MobileViT variants + pretrained weights
# optional fallback (if you want the standalone implementation)
# !pip install -q git+https://github.com/chinhsuanwu/mobilevit-pytorch.git


In [8]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, WeightedRandomSampler
import timm
from collections import Counter
print("Torch:", torch.__version__, "Timm:", timm.__version__)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)


Torch: 2.8.0+cu126 Timm: 1.0.19
Device: cpu


In [10]:
# list available MobileViT-like models in timm
models = timm.list_models('*mobilevit*')
print("MobileViT variants found in timm:", models)

# choose one automatically (you can change this name)
if len(models) == 0:
    raise RuntimeError("No MobileViT models found in timm. You can pip-install a standalone MobileViT repo instead.")
model_name = models[0]   # default to first found; or set e.g. 'mobilevit_xxs' or 'mobilevit_s'
print("Using model:", model_name)



MobileViT variants found in timm: ['mobilevit_s', 'mobilevit_xs', 'mobilevit_xxs', 'mobilevitv2_050', 'mobilevitv2_075', 'mobilevitv2_100', 'mobilevitv2_125', 'mobilevitv2_150', 'mobilevitv2_175', 'mobilevitv2_200']


In [13]:
import timm
import torch
import torch.nn as nn

# 1. Choose model
model_name = "mobilevitv2_050"

# 2. Load pretrained model from timm
model = timm.create_model(model_name, pretrained=True)

# 3. Adapt classifier head for 2 classes (ai, real)
if hasattr(model, "reset_classifier"):
    model.reset_classifier(num_classes=2)
else:
    # fallback if reset_classifier not available
    in_features = model.classifier.in_features
    model.classifier = nn.Linear(in_features, 2)

# 4. Move to device (GPU if available, otherwise CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

print(f"✅ Loaded {model_name} with classifier reset to 2 classes")


✅ Loaded mobilevitv2_050 with classifier reset to 2 classes


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, WeightedRandomSampler
from collections import Counter
from tqdm import tqdm
import copy
import os

# -------------------------------
# 1) Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

# -------------------------------
# 2) Dataset & DataLoader (reuse your train_dataset and test_dataset)
# Handle class imbalance with WeightedRandomSampler
targets = [s[1] for s in train_dataset.samples]
counts = Counter(targets)
class_weights = {cls: 1.0 / count for cls, count in counts.items()}
sample_weights = [class_weights[t] for t in targets]

sampler = WeightedRandomSampler(sample_weights, num_samples=len(sample_weights), replacement=True)

batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, sampler=sampler, num_workers=2, pin_memory=True)
test_loader  = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2, pin_memory=True)

# -------------------------------
# 3) Loss, optimizer, scheduler
criterion = nn.CrossEntropyLoss()  # class indices 0/1
optimizer = optim.AdamW(model.parameters(), lr=2e-4, weight_decay=1e-4)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10)  # adjust T_max for epochs

# Mixed precision
scaler = torch.cuda.amp.GradScaler() if torch.cuda.is_available() else None

# -------------------------------
# 4) Train & Validation functions
def validate(model, loader):
    model.eval()
    running_loss, correct, total = 0.0, 0, 0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            running_loss += loss.item() * images.size(0)
            preds = outputs.argmax(dim=1)
            correct += (preds == labels).sum().item()
            total += images.size(0)
    return running_loss/total, correct/total

def train_one_epoch(model, loader, optimizer, scaler=None):
    model.train()
    running_loss, correct, total = 0.0, 0, 0
    loop = tqdm(loader, desc="Train")
    for images, labels in loop:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        if scaler is not None:
            with torch.cuda.amp.autocast():
                outputs = model(images)
                loss = criterion(outputs, labels)
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
        else:
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
        running_loss += loss.item() * images.size(0)
        preds = outputs.argmax(dim=1)
        correct += (preds == labels).sum().item()
        total += images.size(0)
        loop.set_postfix(loss=running_loss/total, acc=100*correct/total)
    return running_loss/total, correct/total

# -------------------------------
# 5) Training loop with metrics storage
num_epochs = 10
best_acc = 0.0
best_model_wts = copy.deepcopy(model.state_dict())
save_path = "/content/drive/MyDrive/Dataset/mobilevitv2_050_best.pth"

# Lists to store metrics per epoch
train_losses, train_accs = [], []
val_losses, val_accs = [], []

for epoch in range(num_epochs):
    # Train
    train_loss, train_acc = train_one_epoch(model, train_loader, optimizer, scaler)
    # Validate
    val_loss, val_acc = validate(model, test_loader)
    # Scheduler step
    scheduler.step()

    # Store metrics
    train_losses.append(train_loss)
    train_accs.append(train_acc)
    val_losses.append(val_loss)
    val_accs.append(val_acc)

    print(f"Epoch {epoch+1}/{num_epochs}: Train Loss {train_loss:.4f} Acc {train_acc:.4f} | Val Loss {val_loss:.4f} Acc {val_acc:.4f}")

    # Save best model
    if val_acc > best_acc:
        best_acc = val_acc
        best_model_wts = copy.deepcopy(model.state_dict())
        torch.save({
            'epoch': epoch+1,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'best_acc': best_acc
        }, save_path)
        print(f"💾 Best model saved: {save_path}")

# Load best weights at the end
model.load_state_dict(best_model_wts)
print("✅ Training complete, best model loaded.")


Device: cpu


Train:   0%|          | 0/100 [00:00<?, ?it/s]/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
Train: 100%|██████████| 100/100 [35:24<00:00, 21.24s/it, acc=75.2, loss=0.552]


Epoch 1/10: Train Loss 0.5524 Acc 0.7518 | Val Loss 0.2921 Acc 0.8888
💾 Best model saved: /content/drive/MyDrive/Dataset/mobilevitv2_050_best.pth


Train: 100%|██████████| 100/100 [34:20<00:00, 20.61s/it, acc=92.8, loss=0.19]


Epoch 2/10: Train Loss 0.1897 Acc 0.9281 | Val Loss 0.1341 Acc 0.9437
💾 Best model saved: /content/drive/MyDrive/Dataset/mobilevitv2_050_best.pth


...

Train: 100%|██████████| 100/100 [34:34<00:00, 20.74s/it, acc=98.1, loss=0.0603]


Epoch 5/10: Train Loss 0.0603 Acc 0.9806 | Val Loss 0.1065 Acc 0.9563
💾 Best model saved: /content/drive/MyDrive/Dataset/mobilevitv2_050_best.pth


Train: 100%|██████████| 100/100 [34:50<00:00, 20.91s/it, acc=98.7, loss=0.0398]


Epoch 6/10: Train Loss 0.0398 Acc 0.9869 | Val Loss 0.1091 Acc 0.9587
💾 Best model saved: /content/drive/MyDrive/Dataset/mobilevitv2_050_best.pth


Train:  13%|█▎        | 13/100 [05:26<33:13, 22.91s/it, acc=98.6, loss=0.0566]